# 5장 Creating Custom Graph Aggregation Operators

## 이번장의 학습목표 
### 이번장에서는 graph에서 모든 노드들의 이웃정보를 취합하는데 유용한 aggregateMessages 연산자에 대해서 배움.
#### NCAA College Basketball 데이터셋을 aggregateMessages에 적용하여 아래 3가지를 할 수 있음
- aggregateMessages 기본 메카니즘과 패턴을 이해함
- custom한 graph aggregation 연산자을 만들기 위해서 aggregateMessages을 활용해봄.
- aggregateMessages의 성능과 효율성을 최적화 하기

## 01절 NCAA College Basketball datasets

- https://www.packtpub.com/big-data-and-business-intelligence/apache-spark-graph-processing
- 2개의 CSV 데이터셋으로 구성됨.
- teams.csv 는 NCAA Division I competition(미국대학스포츠협회 )에서 참가한 모든 대학팀의 리스트를 포함.  각각의 팀은 4자릿수 ID번호가 부여됨.
```
team_id,team_name
1101,Abilene Chr
1102,Air Force
1103,Akron
1104,Alabama
1105,Alabama A&M
1106,Alabama St
1107,Albany NY
1108,Alcorn St
1109,Alliant Intl
1110,American Univ
```
- stats.csv 는 2014~2015 정규시즌동안의 농구 경기의 점수와 통계치가 포함됨.
```
2015,11,1103,74,1420,57,H,0,25,53,12,30,12,21,9,22,17,12,7,5,22,20,48,3,12,14,28,8,21,13,14,9,0,19
2015,11,1104,82,1406,54,H,0,29,63,7,23,17,19,12,20,17,8,14,9,16,18,50,7,21,11,15,13,19,9,21,2,2,20
2015,11,1112,78,1291,55,H,0,31,54,4,16,12,25,7,28,17,8,11,5,11,20,52,7,21,8,12,3,17,8,16,2,0,17
2015,11,1113,86,1152,50,H,0,30,49,7,16,19,34,8,25,17,11,7,4,22,17,53,3,20,13,23,12,20,7,17,5,3,30
2015,11,1119,84,1102,78,H,0,30,61,13,25,11,17,14,19,18,15,6,4,21,31,62,6,19,10,15,11,12,14,18,8,6,
```

### 데이터셋을 파싱해서 RDD로 읽어오는 모듈을 만들어 보자.
- 농구 경기에 대한 한팀의 통계치를 저장하는 GameStats 클래스를 만들어보자.

In [ ]:
case class GameStats(
    val score: Int,
    val fieldGoalMade: Int,
    val fieldGoalAttempt: Int,
    val threePointerMade: Int,
    val threePointerAttempt: Int,
    val threeThrowsMade: Int,
    val threeThrowsAttempt: Int,
    val offensiveRebound: Int,
    val defensiveRebound: Int,
    val assist: Int,
    val turnOver: Int,
    val steal: Int,
    val block: Int,
    val personalFoul: Int
){
    def fgPercent: Double = 100.0 * fieldGoalMade / fieldGoalAttempt
    def tpPercent: Double = 100.0 * threePointerMade / threePointerAttempt
    def ftPercent: Double = 100.0 * threeThrowsMade / threeThrowsAttempt
    override def toString: String = "Score: " + score 
}

- 경기내에서 효율적인 공격을 어떻게 한지를 알아보기 위해서 GameStats 클래스에 다음의 메소드를 추가

In [ ]:
##  Field Goal percentage
def fgPercent: Double = 100.0 * fieldGoalMade / fieldGoalAttempt
    
## Three Point percentage
def tpPercent: Double = 100.0 * threePointerMade / threePointerAttempt
    
## Free throws percentage
def ftPercent: Double = 100.0 * threeThrowsMade / threeThrowsAttempt
override def toString: String = "Score: " + score

 - game 결과를 위한 클래스 2개를 만듬.
 - FullResult 클래스에는 경기 시즌의 년도와 날짜, 장소가 포함하고, 이긴 팀과 진 팀의 경기에 대한 통계치도 포함.

In [ ]:
#### 
####  중요 : extends Serializable 을 추가 필요함.
####

abstract class GameResult (
    val season: Int,
    val day: Int,
    val loc: String
) extends Serializable

case class FullResult (
    override val season: Int,
    override val day: Int,
    override val loc: String,
    val winnerStats: GameStats,
    val loserStats: GameStats
) extends GameResult(season, day, loc)

 - 정규시즌의 통계 graph을 만들어 보자.
 - 팀들은 node가 되고, edge는 게임 경기가 됨.
 - teams.csv 파일을 파싱해서 team이라는 RDD 변수에 넣어보자.

In [ ]:
bin/spark-shell  --driver-memory 2g  --master local[4]

import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

## ! _.startsWith("#") 를  _.startsWith("team_id") 변경 필요함.
val teams: RDD[(VertexId, String)] =
   sc.textFile("./data/teams.csv").filter(! _.startsWith("team_id")).map { line =>
       val row = line split ','
       (row(0).toInt, row(1))
   }

teams.take(3).foreach{println}

- RDD[Edge[FullResult]] 타임으로 경기결과를 RDD로 로딩함.
- 이긴팀ID, 진팀ID, 2개팀의 경기통계치로 구성됨.

In [ ]:
        val detailedStats: RDD[Edge[FullResult]] =
            sc.textFile("./data/stats.csv").
            filter(! _.startsWith("#")).
            map {line =>
                val row = line split ','
                Edge(row(2).toInt, row(4).toInt, 
                    FullResult(
                        row(0).toInt, row(1).toInt, 
                        row(6),
                        GameStats(      
                                        score = row(3).toInt,
                                fieldGoalMade = row(8).toInt,
                             fieldGoalAttempt = row(9).toInt, 
                             threePointerMade = row(10).toInt,
                          threePointerAttempt = row(11).toInt,   
                              threeThrowsMade = row(12).toInt,
                           threeThrowsAttempt = row(13).toInt, 
                             offensiveRebound = row(14).toInt,
                             defensiveRebound = row(15).toInt,
                                       assist = row(16).toInt,
                                     turnOver = row(17).toInt,
                                        steal = row(18).toInt,
                                        block = row(19).toInt,
                                 personalFoul = row(20).toInt
                        ),
                        GameStats(
                                        score = row(5).toInt,
                                fieldGoalMade = row(21).toInt,
                             fieldGoalAttempt = row(22).toInt, 
                             threePointerMade = row(23).toInt,
                          threePointerAttempt = row(24).toInt,
                              threeThrowsMade = row(25).toInt,
                           threeThrowsAttempt = row(26).toInt, 
                             offensiveRebound = row(27).toInt,
                             defensiveRebound = row(28).toInt,
                                       assist = row(20).toInt,
                                     turnOver = row(30).toInt,
                                        steal = row(31).toInt,
                                        block = row(32).toInt,
                                 personalFoul = row(33).toInt
                        )
                    )
                )
            }
            
detailedStats.take(3).foreach(println)

- 경기 통계 graph를 생성
- 2015 NCAA 우승대학인 Duke 대학이 진 4개의 경기를 조회해보자.

In [ ]:
## Game Score graphs
val fullScoreGraph = Graph(teams, detailedStats)

## Score for 2015 season  
val scoreGraph = fullScoreGraph subgraph (epred = _.attr.season == 2015)

## Print to test
scoreGraph.triplets.filter(_.dstAttr == "Duke").foreach(println)

## 02절 The aggregateMessages operator

### GraphX에 있는 aggregateMessages 연산자를 이용해서 scoreGraph에서 통계데이터를 취합해보자.
- 이긴팀들의 경기당 field goals made 평균 점수을 구해보자.
- 각각의 이긴팀의 평균을 얻기 위해서, 이긴팀들의 field goals made점수를 계산함. 

In [ ]:
// Aggregate the total field goals made by winning teams
type FGMsg = (Int, Int)
val winningFieldGoalMade: VertexRDD[FGMsg] = scoreGraph
aggregateMessages(
    // sendMsg
    triplet => triplet.sendToSrc(1, triplet.attr.winnerStats.fieldGoalMade)
    // mergeMsg
    ,(x, y) => (x._1 + y._1, x._2+ y._2)
)


// Aggregate the total field goals made by winning teams
type Msg = (Int, Int)
type Context = EdgeContext[String, FullResult, Msg]
val winningFieldGoalMade: VertexRDD[Msg] = scoreGraph aggregateMessages(
     // sendMsg
     (ec: Context) => ec.sendToSrc(1, ec.attr.winnerStats.fieldGoalMade),
     // mergeMsg
     (x: Msg, y: Msg) => (x._1 + y._1, x._2+ y._2)
)

### EdgeContext

- 위의 코드를 천천히 보자.
- scoreGraph 객체에서 aggregateMessages 메소드를 호출하였고, 인자로 2개의 함수를 넘겨줌.
- 첫번째 함수의 시그너처는 EdgeContext[VD, ED, Msg] => Unit 이며, EdgeContext을 인자로 받아서 리턴값은 없으며, node에게 message을 보내는 side effect을 생성함.

----------------------------------------------------------

- EdgeContext은 EdgeTriplet과 비슷하게 이웃 노드를 포함한 edge을 표현하며, source노드와 destination노드를 접근할 수 있음.
- EdgeContext은 source노드와 destination노드에게만 message을 보내는 메소드를 갖음. sendToSrc(), sendToDst()
- 각각 triplet에 보내지는 메시지 타임을 정의한 것이 Msg임

----------------------------------------------------------

- Msg객체가 이긴팀인 source 노드에게 전달되고 전달된 실제적인 값은 ( 1,  ec.attr.winnerStats.fieldGoalMade ) 임.
- 그래서, 첫번째 값은 이긴 게임의 수이고, 두번째 값은 득점수를 추출하는 역할을 함.

----------------------------------------------------------

- aggregateMessages의 두번째 인자인 mergeMsg함수는 시그너처 (Msg, Msg) => Msg 을 갖음.
- mergeMsg 함수는 두개의 Msg을 받아서 하나의 Msg을 생성함.
- aggregateMessages은 VertexRDD[Msg]안에 취합된 메시지를 리턴함.

----------------------------------------------------------

- aggregateMessages의 결과를 mapValues()함수를 적용해서 평균을 구함.

In [ ]:
// Average field goals made per Game by winning teams
val avgWinningFieldGoalMade: VertexRDD[Double] =
    winningFieldGoalMade mapValues (
        (id: VertexId, x: Msg) => x match {
            case (count: Int, total: Int) => total.toDouble/count
        }
)
    
avgWinningFieldGoalMade.take(5).foreach(println)

- aggregateMessages와 EdgeContext의 정의는 아래와 같음.

```
class Graph[VD, ED] {
    def aggregateMessages[Msg: ClassTag](
    sendMsg: EdgeContext[VD, ED, Msg] => Unit,
        mergeMsg: (Msg, Msg) => Msg,
        tripletFields: TripletFields = TripletFields.All)
     : VertexRDD[Msg]
}

abstract class EdgeContext[VD, ED, A] {

   // Attribute associated with the edge:
   abstract def attr: ED
   
   // Vertex attribute of the edge's source vertex.
   abstract def srcAttr: VD
   
   // Vertex attribute of the edge's destination vertex.
   abstract def dstAttr: VD
   
   // Vertex id of the edge's source vertex.
   abstract def srcId: VertexId
   
   // Vertex id of the edge's destination vertex.
   abstract def dstId: VertexId
   
   // Sends a message to the destination vertex.
   abstract def sendToDst(msg: A): Unit
   
   // Sends a message to the source vertex.
   abstract def sendToSrc(msg: A): Unit
}

```


## 03절 Abstracting out the aggregation

- 이긴팀의 게임당 득점의 평균을 좀더 간단한 방법으로 구해보자.

In [ ]:
// Aggregate the points scored by winning teams
val winnerTotalPoints: VertexRDD[(Int, Int)] =
scoreGraph.aggregateMessages(
    // sendMsg
    triplet => triplet.sendToSrc(1, triplet.attr.winnerStats.score),
    // mergeMsg
    (x, y) => (x._1 + y._1, x._2+ y._2)
)   

// Average field goals made per Game by winning teams
var winnersPPG: VertexRDD[Double] =
    winnerTotalPoints mapValues (
        (id: VertexId, x: (Int, Int)) => x match {
            case (count: Int, total: Int) => total.toDouble/count
})
    
winnersPPG.take(5).foreach(println)

### Keeping things DRY
- 코치가 이긴 게임에서 3점슛의 평준중에서 가장 높은 5개의 팀의 리스트를 보고 싶어함.
- 이 요구를 좀도 효율적으로 처리하는 방법을 알아보자.

-------------

- 위의 코드를 추상화해서 재사용성있는 코드로 만들어보자.
- 스칼라는 이때 필요한 고차함수들을 지원함.

--------------

- GameStats을 입력받아서 원하는 통계량을 리턴하는 함수를 정의하자.
- 3점 득점과 3점슛의 퍼센트의 평균을 구해보자.

In [ ]:
// Getting individual stats
def threePointMade(stats: GameStats) = {
    stats.threePointerMade
}

def threePointPercent(stats: GameStats) = {
    stats.tpPercent
}

- stats graph 변수와 signature GameStats => Double인 함수를 2개의 인자로 받는 함수를 정의함.

In [ ]:
// Generic function for stats averaging
def averageWinnerStat(graph: Graph[String, FullResult])(getStat: GameStats => Double): VertexRDD[Double] = {
    type Msg = (Int, Double)
    val winningScore: VertexRDD[Msg] =
    graph.aggregateMessages[Msg](
        // sendMsg
        triplet => triplet.sendToSrc(1, getStat(triplet.attr.winnerStats)),
        // mergeMsg
        (x, y) => (x._1 + y._1, x._2+ y._2)
    )
    winningScore mapValues (
        (id: VertexId, x: Msg) => x match {
            case (count: Int, total: Double) => total/count
        })
}
    
    
val winnersThreePointMade = averageWinnerStat(scoreGraph)(threePointMade)
val winnersThreePointPercent = averageWinnerStat(scoreGraph)(threePointPercent)


winnersThreePointMade.sortBy(_._2,false).take(5).foreach(println)
winnersThreePointPercent.sortBy(_._2,false).take(5).foreach(println)

### Coach wants more numbers

- 각각의 팀의 모든 게임의 평균을 얻기를 원함.
- 이전 코드들은 이긴팀에 대한 정보를 구했지만, 진팀에 대한 정보를 구하지 않음. => 이전 코드를 좀더 유연하게 추상화를 해보자.

-----------------

- 이전 코드를 Teams 이라는 추가적인 인자를 고차함수로 수정하자.

In [ ]:
trait Teams
case class Winners extends Teams
case class Losers extends Teams
case class AllTeams extends Teams


def averageStat(graph: Graph[String, FullResult])(getStat: GameStats => Double, tms: Teams): VertexRDD[Double] = {
    type Msg = (Int, Double)
    val aggrStats: VertexRDD[Msg] = graph.aggregateMessages[Msg](
        // sendMsg
        tms match {
            case _ : Winners => t => t.sendToSrc((1, getStat(t.attr.winnerStats)))
            case _ : Losers  => t => t.sendToDst((1,  getStat(t.attr.loserStats)))
            case _           => t => {
                t.sendToSrc((1, getStat(t.attr.winnerStats)))
                t.sendToDst((1, getStat(t.attr.loserStats)))
            }
        }
    ,
    // mergeMsg
    (x, y) => (x._1 + y._1, x._2+ y._2)
    )
        
    aggrStats mapValues (
        (id: VertexId, x: Msg) => x match {
            case (count: Int, total: Double) => total/count
    })
}
    
    
## Average Three Point Made Per Game for All Teams
val allThreePointMade = averageStat(scoreGraph)(threePointMade, AllTeams())    
allThreePointMade.sortBy(_._2, false).take(5).foreach(println)

## 진게임에서 3점수의 평균
val loseThreePointMade = averageStat(scoreGraph)(threePointMade, Losers())    
loseThreePointMade.sortBy(_._2, false).take(5).foreach(println)

## Average Three Point Percent for All Teams
val allThreePointPercent = averageStat(scoreGraph)(threePointPercent, AllTeams())
allThreePointPercent.sortBy(_._2,false).take(5).foreach(println)

### Calculating average points per game

- 가장 많은 점수와 가장 적은 점수로 이긴 2팀을 찾아보자.

In [ ]:
// Winning teams
def score(stats: GameStats) = {
    stats.score
}
val winnerAvgPPG = averageStat(scoreGraph)(score, Winners())

winnerAvgPPG.max()(Ordering.by(_._2))
winnerAvgPPG.min()(Ordering.by(_._2))

- 모든 경기에서 가장 공격적인 팀과 공격적인 나쁜 팀을 찾아보자.

In [ ]:
val allAvgPPG = averageStat(scoreGraph)(score, AllTeams())
allAvgPPG.max()(Ordering.by(_._2))
allAvgPPG.min()(Ordering.by(_._2))

### Defense stats – D matters as in direction

- 각각의 팀은 상대방팀에게 허용하는 평균득점수와 리바운드수는 얼마인가??
- 이것을 알아보기 위해서 averageConcededStat() 고차함수를 정의함.
- 이긴팀에 대해서는 loserStats를 전달하고, 진팀에 대해서는 winnerStats를 전달함.

In [ ]:
def averageConcededStat(graph: Graph[String, FullResult])(getStat:GameStats => Double, rxs: Teams): VertexRDD[(String, Double)] = {
    type Msg = (Int, Double, String)
    val aggrStats: VertexRDD[Msg] = graph.aggregateMessages[Msg](
        // sendMsg
        rxs match {
            case _ : Winners => t => t.sendToSrc((1, getStat(t.attr.loserStats), t.srcAttr))
            case _ : Losers  => t => t.sendToDst((1, getStat(t.attr.winnerStats), t.dstAttr))
            case _           => t => {
                t.sendToSrc((1, getStat(t.attr.loserStats),t.srcAttr))
                t.sendToDst((1, getStat(t.attr.winnerStats),t.dstAttr))
            }
        }
        ,
        // mergeMsg
        (x, y) => (x._1 + y._1, x._2+ y._2, x._3)
    )
    aggrStats mapValues (
        (id: VertexId, x: Msg) => x match {
            case (count: Int, total: Double, name: String) =>
            (name, total/count)
        })
}
    
val winnersAvgConcededPoints = averageConcededStat(scoreGraph)(score, Winners())
val losersAvgConcededPoints = averageConcededStat(scoreGraph)(score, Losers())


losersAvgConcededPoints.min()(Ordering.by(_._2))
winnersAvgConcededPoints.min()(Ordering.by(_._2))

losersAvgConcededPoints.max()(Ordering.by(_._2))
winnersAvgConcededPoints.max()(Ordering.by(_._2))

### Joining average stats into a graph
- 각각의 팀에 대해서 더 많은 통계치를 구하고 노드들에 대한 정보를 연결해보자.

----------
- 팀에 대한 통계 클래스를 만드는것부터 시작하자.

In [ ]:
// Average Stats of All Teams
case class TeamStat(
    wins: Int = 0 // Number of wins
    ,losses: Int = 0 // Number of losses
    ,ppg: Int = 0 // Points per game
    ,pcg: Int = 0 // Points conceded per game
    ,fgp: Double = 0 // Field goal percentage
    ,tpp: Double = 0 // Three point percentage
    ,ftp: Double = 0 // Free Throw percentage
    ){
    override def toString = wins + "-" + losses
}

- aggregateMessages을 사용해서 모든 팀에 대한 통계치를 구할 수 있음.
- 8개의 값을 갖는 메시지를 정의하고 경기수, 이긴 경기수, 진 경기수, 통계치로 정의함.

In [ ]:
type Msg = (Int, Int, Int, Int, Int, Double, Double, Double)

val aggrStats: VertexRDD[Msg] =
scoreGraph.aggregateMessages(
    // sendMsg
    t => {
        t.sendToSrc(( 1,
            1, 0,
            t.attr.winnerStats.score,
            t.attr.loserStats.score,
            t.attr.winnerStats.fgPercent,
            t.attr.winnerStats.tpPercent,
            t.attr.winnerStats.ftPercent
            ))
        t.sendToDst(( 1,
            0, 1,
            t.attr.loserStats.score,
            t.attr.winnerStats.score,
            t.attr.loserStats.fgPercent,
            t.attr.loserStats.tpPercent,
            t.attr.loserStats.ftPercent
            ))
        }
    ,
    // mergeMsg
    (x, y) => ( x._1 + y._1, x._2 + y._2,
        x._3 + y._3, x._4 + y._4,
        x._5 + y._5, x._6 + y._6,
        x._7 + y._7, x._8 + y._8
    )
)

- aggregate message로 구한 aggrStats RDD을 TeamStats으로 변경함.

In [ ]:
val teamStats: VertexRDD[TeamStat] = aggrStats mapValues {
    (id: VertexId, m: Msg) => m match {
        case ( 
            count: Int,
            wins: Int,
            losses: Int,
            totPts: Int,
            totConcPts: Int,
            totFG: Double,
            totTP: Double,
            totFT: Double) => TeamStat( wins, losses,
                totPts/count,
                totConcPts/count,
                totFG/count,
                totTP/count,
                totFT/count)
    }
}

- graph에 teamStats을 연결해보자.
- vertex의 속성으로 Team클래스를 정의하고 Team은 이름과 TeamStat Option을 멤버변수로 갖음.

In [ ]:
case class Team(name: String, stats: Option[TeamStat]) {
    override def toString = name + ": " + stats
}

- 이전 장에서 본 joinVertices연산자를 사용함.

In [ ]:
// Joining the average stats to vertex attributes
def addTeamStat(id: VertexId, t: Team, stats: TeamStat) =
Team(t.name, Some(stats))

val statsGraph: Graph[Team, FullResult] =
    scoreGraph.mapVertices((_, name) => Team(name, None)).
        joinVertices(teamStats)(addTeamStat)

In [ ]:
statsGraph.vertices.take(3).foreach(println)

- 정규시즌에서 상위 10개 팀을 뽑아보자.
- Option[TeamStat] 객체를 위한 Ordering option 객체를 정의함.

In [ ]:
import scala.math.Ordering
object winsOrdering extends Ordering[Option[TeamStat]] {
    def compare(x: Option[TeamStat], y: Option[TeamStat]) =
    (x, y) match {
        case (None, None)       => 0
        case (Some(a), None)    => 1
        case (None, Some(b))    => -1
        case (Some(a), Some(b)) => {
            if (a.wins == b.wins)  a.losses compare b.losses
            else  a.wins compare b.wins
        }
    }
}

In [ ]:
import scala.reflect.classTag
import scala.reflect.ClassTag
statsGraph.vertices.sortBy(v => v._2.stats,false)(winsOrdering, classTag[Option[TeamStat]]).take(10).foreach(println)

## 03절 Performance optimization

- aggregateMessages함수의 세번째 인자는 TripletFields을 갖으며, 이것은 EdgeContext에 대한 접근을 어떻게 할지를 지정해줌.
- 기본값은 TripletFields.All으로 EdgeContext 클래스의 모든 필드를 접근 가능함.
    - TripletFields.All: This option exposes all the fields (source, edge, and destination)
    -  TripletFields.Dst: This one exposes the destination and edge fields but not the source field
    -  TripletFields.EdgeOnly: This option exposes only the edge field but not the source or destination field
    -  TripletFields.None: With this option none of the triplet fields are exposed
    -  TripletFields.Src: This one exposes the source and edge fields but not the destination field

 - 만약, 각각의 팀에 대해서 이긴 경기수와 진 경기수에만 관심이 있다면, EdgeContext classd의 필드에는 접근이 필요없기 때문에, TripletFields.None 으로 지정할 수 있음.

In [ ]:
// Number of wins of the teams
val numWins: VertexRDD[Int] = scoreGraph.aggregateMessages(
    triplet => {
        triplet.sendToSrc(1) // No attribute is passed but an integer
    },
    (x, y) => x + y,
    TripletFields.None
)
    
// Number of losses of the teams
val numLosses: VertexRDD[Int] = scoreGraph.aggregateMessages(
    triplet => {
        triplet.sendToDst(1) // No attribute is passed but an integer
    },
    (x, y) => x + y,
    TripletFields.None
)
    
numWins.sortBy(_._2,false).take(5).foreach(println)

numLosses.sortBy(_._2, false).take(5).foreach(println)

- 상위 5개 팀의 이름을 알고 싶으면, srcAttr속성에 접근이 필요하므로, TripletFields.Src로 설정함.

In [ ]:
val numWinsOfTeams: VertexRDD[(String, Int)] = scoreGraph.aggregateMessages(
    t => {
        t.sendToSrc(t.srcAttr, 1) // Pass source attribute   only
    },
    (x, y) => (x._1, x._2 + y._2),
    TripletFields.Src
)
    
numWinsOfTeams.sortBy(_._2._2, false).take(5).foreach(println)
numWinsOfTeams.sortBy(_._2._2).take(5).foreach(println)

### The MapReduceTriplets operator 

- Spark 1.2이전에는 aggregateMessages연산자가 없었고 대신에 mapReduceTriplets연산자가 원시적인 취합 연산자가 였음.
```
class Graph[VD, ED] {
    def mapReduceTriplets[Msg](
        map: EdgeTriplet[VD, ED] => Iterator[(VertexId, Msg)],
        reduce: (Msg, Msg) => Msg) : VertexRDD[Msg]
}
```

- mapReduceTriplets비해서 aggregateMessages연산자가 더 표현력이 풍부하고, TripletFields을 지정할수 있어서 성능을 최적화 할수 있음.